In [1]:
import numpy as np
import pandas as pd

from scipy.stats import chi2_contingency,ttest_ind
from scipy import stats
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import auc, roc_curve
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['font.sans-serif'] = ['FangSong'] # 指定默认字体
matplotlib.rcParams['axes.unicode_minus'] = False # 解决保存图像是负号'-'显示为方块的问题

import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn import ensemble

data1 = pd.read_excel("C:/Users/Ca1/Desktop/papers/Nocturnal_hypertension by C/data1.xlsx")
data2 = pd.read_excel("C:/Users/Ca1/Desktop/papers/Nocturnal_hypertension by C/data2.xlsx")

In [2]:
data1.rename(columns = {'β2-微球蛋白':'B2-微球蛋白','吸烟(0否1是）':'吸烟0否1是','夜间尿量ml':'夜间尿量ml/24H'
                        ,'白天尿量ml':'白天尿量ml/24H','24h-DBP':'24hDBP','是否应用降压药':'是否用降压药','身高':'身高cm'}, inplace = True)
col = data1.columns
data = pd.concat([data1, data2],ignore_index=True)[col]
data=data.drop(data.index[data[data['夜间SBP'] == '夜间数据'].index])#“无”字被去掉了
data=data.drop(data.index[data[data['夜间SBP'] == '夜间有效值不足'].index])#删除五例’夜间SBP’ 无法判断的
data['nocturnal_hypertension'] = (data['夜间SBP']>=120) | (data['夜间DBP']>=70)
data['daytime_hypertension'] = (data['白天SBP'] >=135) | (data['白天DBP']>=85)
data['NH_continuous'] = data['nocturnal_hypertension'] & data['daytime_hypertension']
data['NH_onlynight'] = data['nocturnal_hypertension'] &(~data['daytime_hypertension'])

data.replace({'无':np.nan}, inplace = True)
data.replace({'未查':np.nan}, inplace = True)
data.replace({'':np.nan}, inplace = True)
data.replace({' ':np.nan}, inplace = True)
data['24h-平均压'].replace({'94..9':94.9}, inplace = True)#数据（三）
data['同型半胱氨酸'].replace({'>50':60},inplace = True)#同型半胱氨>50 记为整型60
data['C反应蛋白'].replace({'<0.499':0.01},inplace = True)
data['甘油三酯'].replace({'（外院行检查后拒绝再次检查）':np.nan},inplace = True)
data['尿蛋白/肌酐(mg/g)'].replace({'未查（月经期）':np.nan},inplace = True)
data['24H尿蛋白mg'].replace({'未查':np.nan,'21286.4mg':21286.4,'10397.04':10397.04},inplace = True)
data['空腹血糖'].replace({34.4:4.4, 44.9:4.9}, inplace = True)
data = data.drop(data[data['是否用降压药'] == 10].index)
data = data.drop(data[data['CCB'] == 2].index)

data.rename(columns = {'姓名':'Name','编号':'No.','性别（1男2女）':'Male','年龄':'Age,y','BMI':'BMI,kg/m2','吸烟0否1是':'Smoker'
                       ,'饮酒':'Alcohol intake','糖尿病':'Diabetes mellitus','高血压':'Hypertension','高脂血症':'Hyperlipidemia'
                       ,'高血压家族史':'Hypertension family history','CVD既往史':'CVD history','是否用降压药':'Antihypertension drugs'
                       ,'RAAS药':'RAAS drugs intake','nRAAS药':'nRAAS drugs intake','24H尿蛋白mg':'Proteinuria,mg/24h'
                       ,'HGB':'Hemoglobin,g/L','尿素氮':'BUN,mmol/L','肌酐':'Scr,mmol/L','尿酸':'Uric acid,mmol/L'
                       ,'空腹血糖':'FBG,mmol/L','甘油三酯':'Triglyceride,mmol/L','胆固醇':'Cholesterol,mmol/L','高密度脂蛋白':'HDL-C,mmol/L'
                       ,'低密度脂蛋白':'LDL-C,mmol/L','白蛋白':'Serum albumin,g/L','Na':'Serum sodium,mmol/L','CA':'Serum calcium,mmol/L'
                       ,'P':'Serum phosphate,mmol/L','B2-微球蛋白':'β2-MG,ug/mL','甲状旁腺素（pg/ml）':'iPTH,pg/mL'
                       ,'同型半胱氨酸':'Homocysteine,umol/L','GFR-EPI':'eGFR,mL/ (min·1.73 m2)'
                       ,'诊室SBP':'Clinic SBP,mmHg','诊室DBP':'Clinic DBP,mmHg'},inplace = True)

data = data[['Name','No.','Male','Age,y','BMI,kg/m2','Smoker','Alcohol intake','Diabetes mellitus','Hypertension','Hyperlipidemia'
,'Hypertension family history','CVD history','Antihypertension drugs','RAAS drugs intake','nRAAS drugs intake'
,'Proteinuria,mg/24h','Hemoglobin,g/L','BUN,mmol/L','Scr,mmol/L','Uric acid,mmol/L','FBG,mmol/L','Triglyceride,mmol/L'
,'Cholesterol,mmol/L','HDL-C,mmol/L','LDL-C,mmol/L','Serum albumin,g/L','Serum sodium,mmol/L','Serum calcium,mmol/L'
,'Serum phosphate,mmol/L','β2-MG,ug/mL','iPTH,pg/mL','Homocysteine,umol/L','eGFR,mL/ (min·1.73 m2)'
,'Clinic SBP,mmHg','Clinic DBP,mmHg','nocturnal_hypertension']]


d:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  after removing the cwd from sys.path.


In [4]:
dataplot = data[['Male','Age,y','BMI,kg/m2','Smoker','Alcohol intake','Diabetes mellitus','Hypertension','Hyperlipidemia'
,'Hypertension family history','CVD history','Antihypertension drugs','RAAS drugs intake','nRAAS drugs intake'
,'Proteinuria,mg/24h','Hemoglobin,g/L','BUN,mmol/L','Scr,mmol/L','Uric acid,mmol/L','FBG,mmol/L','Triglyceride,mmol/L'
,'Cholesterol,mmol/L','HDL-C,mmol/L','LDL-C,mmol/L','Serum albumin,g/L','Serum sodium,mmol/L','Serum calcium,mmol/L'
,'Serum phosphate,mmol/L','β2-MG,ug/mL','iPTH,pg/mL','Homocysteine,umol/L','eGFR,mL/ (min·1.73 m2)'
,'Clinic SBP,mmHg','Clinic DBP,mmHg','nocturnal_hypertension']]

dummy_col = [col for col in dataplot.columns if len(dataplot[col].unique()) < 5]
float_col = [col for col in dataplot.columns if col not in dummy_col]

# for col in float_col:
#     f,ax=plt.subplots(1,2,figsize=(12,5)) 
#     ax[0].boxplot(x=dataplot.loc[dataplot[col].isnull() == False,col].values)
#     #columns列索引，values所有数值
#     ax[0].grid()
#     bins = np.linspace(min(dataplot[col].values),max(dataplot[col].values),20)
#     ax[1].hist(dataplot[col].values,bins)
#     ax[1].grid()
#     plt.xlabel(col,fontsize=15)
#     fig = plt.gcf() #需要先获取，才能savefig
#     plt.show()

In [5]:
dummy_col = [col for col in dataplot.columns if len(dataplot[col].unique()) < 5]
float_col = [col for col in dataplot.columns if col not in dummy_col]
for col in float_col:
    data.loc[data[col]>data[col].mean()+3*data[col].std(),col] = np.nan
data=data.fillna(data.mean())
# data[float_col].describe()

In [6]:
dataplot = data[['Male','Age,y','BMI,kg/m2','Smoker','Alcohol intake','Diabetes mellitus','Hypertension','Hyperlipidemia'
,'Hypertension family history','CVD history','Antihypertension drugs','RAAS drugs intake','nRAAS drugs intake'
,'Proteinuria,mg/24h','Hemoglobin,g/L','BUN,mmol/L','Scr,mmol/L','Uric acid,mmol/L','FBG,mmol/L','Triglyceride,mmol/L'
,'Cholesterol,mmol/L','HDL-C,mmol/L','LDL-C,mmol/L','Serum albumin,g/L','Serum sodium,mmol/L','Serum calcium,mmol/L'
,'Serum phosphate,mmol/L','β2-MG,ug/mL','iPTH,pg/mL','Homocysteine,umol/L','eGFR,mL/ (min·1.73 m2)'
,'Clinic SBP,mmHg','Clinic DBP,mmHg','nocturnal_hypertension']]

# dataplot.info()
# dummy_col = [col for col in dataplot.columns if len(dataplot[col].unique()) < 5]
# float_col = [col for col in dataplot.columns if col not in dummy_col]
# for col in float_col:
#     f,ax=plt.subplots(1,2,figsize=(12,5)) 
#     ax[0].boxplot(x=dataplot.loc[dataplot[col].isnull() == False,col].values)
#     #columns列索引，values所有数值
#     ax[0].grid()
#     bins = np.linspace(min(dataplot[col].values),max(dataplot[col].values),20)
#     ax[1].hist(dataplot[col].values,bins)
#     ax[1].grid()
#     plt.xlabel(col,fontsize=15)
#     fig = plt.gcf() #需要先获取，才能savefig
#     plt.show()

In [7]:
data.to_csv('C:/Users/Ca1/Desktop/papers/Nocturnal_hypertension by C//data_continuous_gbk_20201211.csv', encoding= 'gbk', index ='False')